# 🗺 **MAPLAB**

In this notebook, you will be working with freely available real-world mapping data to solve the realistic large problem of finding the shortest (or fastest) path between two points on a map. You will implement the backend for a route-finding application (for example, Google maps), which we will be able to use to plan paths around Stanford University and the Bay Area.

All of the software and data we are working with in this lab is freely available, including the mapping data, the images used to render the maps, and the software used to control the map display! In fact, a lot of the world's most widely used software is created in this same spirit of sharing and community.

# 🚚 **The Data**
The data we'll use for this lab comes from [OpenStreetMap](https://en.wikipedia.org/wiki/OpenStreetMap), and for the bulk to this lab, we are working with data about the Bay Area in California (about 750 MB of data). While the raw data ([downloaded from here](https://download.bbbike.org/osm/)) was specified in the OSM XML Format, we have done a little bit of pre-processing to convert the data to a format that is slightly easier to work with.

As in the original format, we have divided the data into two separate pieces: a list of "nodes" (representing individual locations) and a list of "ways" (representing roads or other kinds of connections between nodes). We've changed it up a bit from that. In it's current format, our data is stored in two dictionaries: 
* a **nodes** dictionary that maps **node IDs** to a tuple of that node's **(latitude, longitude)**
* a **ways** dictionary that maps **node IDs** to a **list of node IDs** representing neighboring nodes.

We have two different data sets: a small one meant for testing and making sure your things are working with a few nodes on the Stanford University campus, and a larger one encompassing much of the Bay Area Peninsula.

In [ ]:
#@title Run this to import your data (and a few functions)!
import pickle
import gdown
from math import acos,cos,sin,pi,atan2

!gdown -q 1_sSOAz9NHTYDDDZSL035ERUDSBQqNLSw
!gdown -q 1iLIl3vJovgMDq4jEfmPjR6k-Viyug73c

def great_circle_distance(loc1, loc2):
    """
    Returns approx distance between (lat1, lon1) and (lat2, lon2) in
    miles, taking into account Earth's curvature (assuming a spherical
    earth).

    Latitude / longitudes in degrees.
    """
    lat1, lon1 = loc1
    lat2, lon2 = loc2
    phi1 = lat1*pi/180.
    theta1 = lon1*pi/180.
    phi2 = lat2*pi/180.
    theta2 = lon2*pi/180.
    cospsi = sin(phi1)*sin(phi2) + cos(phi1)*cos(phi2)*cos(theta2-theta1)
    sinpsi = ((sin(theta1)*cos(phi1)*sin(phi2) - sin(theta2)*cos(phi2)*sin(phi1))**2 +\
              (cos(theta2)*cos(phi2)*sin(phi1) - cos(theta1)*cos(phi1)*sin(phi2))**2 +\
              (cos(phi1)*cos(phi2)*sin(theta2-theta1))**2)**0.5
    return atan2(sinpsi,cospsi) * 3958


def read_osm_data(filename):
    """
    Yield elements from the given filename, which is assumed to contain a
    series of pickled[1] Python objects, stacked end-to-end.

    This structure allows reading the large structures necessary for lab 2
    without loading the entire file into memory at once, and should be much
    faster than reading directly from a .osm file.

    Example usage:
        for element in read_osm_data('some_file'):
            print(element)

    [1] see https://docs.python.org/3/library/pickle.html
    """
    with open(filename, 'rb') as f:
        while True:
            try:
                yield pickle.load(f)
            except EOFError:
                break

ALLOWED_HIGHWAY_TYPES = {
    'motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'unclassified',
    'residential', 'living_street', 'motorway_link', 'trunk_link',
    'primary_link', 'secondary_link', 'tertiary_link',
}

DEFAULT_SPEED_LIMIT_MPH = {
    'motorway': 60,
    'trunk': 45,
    'primary': 35,
    'secondary': 30,
    'residential': 25,
    'tertiary': 25,
    'unclassified': 25,
    'living_street': 10,
    'motorway_link': 30,
    'trunk_link': 30,
    'primary_link': 30,
    'secondary_link': 30,
    'tertiary_link': 25,
}


def to_kml(path):
    """
    Given a path as a list of (latitude, longitude) tuples, return a string
    containing a KML[1] representation of the path, for use with the web
    viewer.

    [1] see https://en.wikipedia.org/wiki/Keyhole_Markup_Language
    """
    out = ("""<?xml version="1.0" encoding="utf-8"?>
<kml xmlns="http://earth.google.com/kml/2.1">
  <Document>
    <Placemark>
      <LineString>
        <extrude>1</extrude>
        <tessellate>1</tessellate>
        <coordinates>""")
    out += " ".join("%f,%f" % (loc[::-1]) for loc in path)
    return out + ("""</coordinates>
      </LineString>
    </Placemark>
    <Placemark>
      <name>Start</name>
      <Point>
        <coordinates>%f,%f,0</coordinates>
      </Point>
    </Placemark>
    <Placemark>
      <name>End</name>
      <Point>
        <coordinates>%f,%f,0</coordinates>
      </Point>
    </Placemark>
  </Document>
</kml>""" % ((path[0][::-1]) + (path[-1][::-1])))

def add_way(cns, connections, nodes, speed):
    for n in range(len(nodes)-1):
        if nodes[n] in connections:
            connections[nodes[n]].append((nodes[n+1], speed))
            cns[nodes[n]].append(nodes[n+1])
        else:
            connections[nodes[n]] = [(nodes[n+1], speed)]
            cns[nodes[n]]=[nodes[n+1]]
    if nodes[-1] not in connections:
        connections[nodes[-1]] = []
        cns[nodes[-1]] = []

def build_auxiliary_structures(nodes_filename, ways_filename):
    """
    Creates an internal representation you you want for the specified map, by
    reading the data from the given filenames (using read_osm_data)
    """
    connections = {}
    cns = {}
    for way in read_osm_data(ways_filename):
        if 'highway' in way['tags'] and way['tags']['highway'] in ALLOWED_HIGHWAY_TYPES:
            if 'maxspeed_mph' in way['tags']:
                speed = way['tags']['maxspeed_mph']
            else:
                speed = DEFAULT_SPEED_LIMIT_MPH[way['tags']['highway']]
            if 'oneway' not in way['tags'] or way['tags']['oneway']=='no':
                add_way(cns, connections, way['nodes'][::-1], speed)
            add_way(cns, connections, way['nodes'], speed)

    nodes = {}
    special_names = {}
    for node in read_osm_data(nodes_filename):
        if node['id'] in connections:
            nodes[node['id']] = (node['lat'], node['lon'])
            if 'name' in node['tags']:
                special_names[node['id']]=node['tags']['name']
    return [nodes, connections, cns, special_names]

stanford_speed_ways = {1:[(7, 35), (2, 25), (3, 35)], 2:[(1, 25), (8, 15), (6, 15)], 3:[(1, 25), (4, 30), (10, 35)], 4:[(3, 30)], 5:[(6, 15), (8, 15), (9, 15)], 6:[(2, 15), (5, 15), (10, 15)], 7:[(1, 35), (9, 35)], 8:[(2, 15), (5, 15)], 9:[(5, 15), (7, 35)], 10:[(6, 15), (3, 35)]}
stanford_ways = {1:[7, 2, 3], 2:[1, 8, 6], 3:[1, 4, 10], 4:[3], 5:[6, 8, 9], 6:[2, 5, 10], 7:[1, 9], 8:[2, 5], 9:[5, 7], 10:[6, 3]}
bay_nodes, bay_speed_ways, bay_ways, special_names = build_auxiliary_structures('bayarea.nodes', 'bayarea.ways')
stanford_nodes = {1:(37.434525, -122.167895), 2:(37.429852, -122.169461), 3:(37.431544, -122.164219), 4:(37.434426, -122.161007), 5:(37.424908, -122.169030), 6:(37.427806, -122.166914), 7:(37.433323, -122.175912), 8:(37.427484, -122.170277), 9:(37.424012, -122.179842), 10:(37.427956,-122.1611884)}

## 🌲 **Stanford Data**
Let's take a look at what our Stanford data looks like below. Take a look at the image to see the nodes and connections between those nodes.

If you know the Stanford map well, you'll notice these nodes are at specific locations around the campus.

>  Node 1: Palm Drive (near entrance)

>  Node 2: the Oval

>  Node 3: Visitor Center

>  Node 4: Football Stadium

>  Node 5: Bookstore

>  Node 6: Hoover Tower

>  Node 7: Stanford Hospital

>  Node 8: the Quad

>  Node 9: the driving range

>  Node 10: Stanford GSB (Graduate School of Business)

<img src="https://drive.google.com/uc?export=view&id=10jG51Rq0NpOjdjc8OLJeY13m9YRO0zu-" 
     width="1000" 
     height="auto" />



### 💡 **Check yourself!**

Assuming there are no path lengths for this map, what is the shortest path  from the football stadium (node 4) to the driving range (node 9)?

In [ ]:
# FILL IN THE PATH BELOW
shortest_path = [4,3,1,7,9]

In [ ]:
#@title Run this to check your answer!
if shortest_path == [4, 3, 1, 7, 9]:
  print("That's right!")
else:
  print("That's not the correct path.")

That's right!


### 💡 **Check yourself!**

What are the neighbors of node 1?

In [ ]:
# FILL IN THE NEIGHBORS BELOW
node1_neighbors = [7,3,2]

In [ ]:
#@title Run this to check your answer!
if set(node1_neighbors) == {7, 2, 3}:
  print("That's right!")
else:
  print("Those aren't the correct neighbors.")

That's right!


## 🧰 **Creating Stanford Data**

The **nodes** dictionary has already been created for you! Check it out by running the cell below.

In [ ]:
stanford_nodes

{1: (37.434525, -122.167895),
 2: (37.429852, -122.169461),
 3: (37.431544, -122.164219),
 4: (37.434426, -122.161007),
 5: (37.424908, -122.16903),
 6: (37.427806, -122.166914),
 7: (37.433323, -122.175912),
 8: (37.427484, -122.170277),
 9: (37.424012, -122.179842),
 10: (37.427956, -122.1611884)}

It's up to you to create the corresponding **ways** dictionary. Remember, this dictionary should map **node IDs** to a **list** of neighboring **node IDs**. Fill in the dictionary below with the correct information based on the image above.

In [ ]:
stanford_ways = {1:[2,3,7],2:[1,6,8],3:[1,4,10],4:[3],5:[6,8,9],6:[2,5,10],7:[1,9],8:[2,5],9:[5,7],10:[3,6]} # FILL THIS IN!

In [ ]:
#@title Run this to check your solution!
incorrect = False
for i in range(1, 11):
  if i not in stanford_ways:
    print("Your ways dictionary is missing node", i, ".")
    incorrect = True

if set(stanford_ways[1])!={7, 2, 3}:
  print("Your neighbors for node 1 are not correct.")
elif set(stanford_ways[2])!={1, 8, 6}:
  print("Your neighbors for node 2 are not correct.")
elif set(stanford_ways[3])!={1, 4, 10}:
  print("Your neighbors for node 3 are not correct.")
elif set(stanford_ways[4])!={3}:
  print("Your neighbors for node 4 are not correct.")
elif set(stanford_ways[5])!={6, 8, 9}:
  print("Your neighbors for node 5 are not correct.")
elif set(stanford_ways[6])!={2, 5, 10}:
  print("Your neighbors for node 6 are not correct.")
elif set(stanford_ways[7])!={1, 9}:
  print("Your neighbors for node 7 are not correct.")
elif set(stanford_ways[8])!={2, 5}:
  print("Your neighbors for node 8 are not correct.")
elif set(stanford_ways[9])!={5, 7}:
  print("Your neighbors for node 9 are not correct.")
elif set(stanford_ways[10])!={6, 3}:
  print("Your neighbors for node 10 are not correct.")
elif not incorrect:
  print("You have the right ways dictionary!")

You have the right ways dictionary!


## 🌉 **Bay Area Data**
The bay area data is stored in the variables `bay_nodes` and `bay_ways`. Use this data to answer the questions below.

### 🛑 **How many nodes are in the Bay Area data set?**

In [ ]:
# use this cell to answer the question!

bay_total_nodes = len(bay_nodes)

In [ ]:
#@title Run this to check your answer!
if bay_total_nodes == 286589:
  print("That's the right number of nodes!")
else:
  print("That's NOT the right number of nodes.")

That's the right number of nodes!


### 🛑 **What is the longitude and latitude tuple for node 6948994470?**

In [ ]:
# use this cell to answer the question!

loglat_tup = bay_nodes[6948994470]

In [ ]:
#@title Run this to check your answer!
if loglat_tup == (37.7689175, -122.4980223):
  print("That's the right tuple!")
else:
  print("That's NOT the right tuple.")

That's the right tuple!


Try putting this tuple into google maps! If you go to https://www.google.com/maps/ then put **(longitude, latitude)** in the search bar, it'll take you to where the location actually is in the world!

In [ ]:
#@title What location is this?
location_in_real_world = "San Francisco" #@param {type:"string"}


<font color="#de3023"><h1><b>STOP #1: Raise your hand to check in with your instructor before moving on.</b></h1></font>


# 🚗 **Path Planning!**
Now that we know our data, let's do this! We're no longer doing unweighted search - since we're using a map, our nodes have real distances between them! 


## ⏸ **A Brief Interlude on Sorting**
In class, we haven't talked about how to sort a list by a specific value. Let's explore how to use the function `sort` to do this!

First, check out the link to [this documentation here](https://docs.python.org/3/howto/sorting.html). Take a look at the **Sorting Basics** and **Key Functions** section. We're going to be using the `sort` function, which you use like:
```python
list_name.sort()
```
`sort` also takes in two optional parameters: 

With `reverse`, you can choose if you want the sorted list to be ascending or descending order. Here's how you could set to `True`:
```python
list_name.sort(reverse=True)
```
If you set `reverse` to `True`, the sorted list will be in **descending** order, aka high value to low value. If you set it to `False` or don't set it at all, the sorted list will be in **ascending order**, aka low value to high value.

With `key`, you can choose **how** to sort the list. If you don't want to be sorting the list by the values in the list themselves and instead by something else, you can use this. 

The format is something you're probably not used to. It uses what's called an **inline** or **lambda** function. We're not going to go into this a ton, but you can use the examples below and in the docs as a guide.

Here are some examples of using the `key` parameter:


```python
a = [1, 6, 3, 8, 2]
b = [(1,5), (7,3), (4,6), (3,2)]
c = {1:10, 3:4, 6:2, 8:9, 2:1, 7:8, 4:10}

a.sort(reverse=False) # regular sort
>>> [1, 2, 3, 6, 8]

b.sort(reverse=False, key=lambda x:x[1]) # sorting by the second value in the tuple
>>> [(3,2), (7,3), (1,5), (4,6)]

a.sort(reverse=False, key=lambda x:c[x]) # sorting by the value in the dict c
>>> [2, 6, 3, 8, 1]
```






### 📎 **What is the order of the list `[(1,5), (7,3), (4,6), (3,2)]` if we sort it ascending by the first value in the tuple?**

In [ ]:
# use this cell to answer the question!

sortlist_order = [(1,5),(3,2),(4,6),(7,3)]

In [ ]:
#@title Run this to check your answer!
if sortlist_order == [(1,5), (3,2), (4,6), (7,3)]:
  print("That's the right sorted list!")
else:
  print("That's NOT the right sorted list.")

That's the right sorted list!


### 📎 **Now use the sort function in the cell below to sort `[(1,5), (7,3), (4,6), (3,2)]` ascending by the first value in the tuple.**

In [ ]:
# use sort on this list to get the list you came up with above!

sortfn_list_order = [(1,5), (7,3), (4,6), (3,2)]
sortfn_list_order.sort(reverse=False, key=lambda x:x[0])

In [ ]:
#@title Run this to check your answer!
if sortfn_list_order == [(1,5), (3,2), (4,6), (7,3)]:
  print("That's the right sorted list!")
else:
  print("That's NOT the right sorted list.")

That's the right sorted list!


### 📎 **What is the order of the list `[(1,5), (7,3), (4,6), (3,2)]` if we sort it ascending by the first value in the tuple's value in the dictionary `c`?**

#### `c = {1:10, 3:4, 6:2, 8:9, 2:1, 7:8, 4:10}`

In [ ]:
# use this cell to answer the question!
sort_wdict = [(1,5),(4,6),(7,3),(3,2)]

In [ ]:
#@title Run this to check your answer!
if sort_wdict == [(1, 5), (4, 6), (7, 3), (3, 2)]:
  print("That's the right sorted list!")
else:
  print("That's NOT the right sorted list.")

That's the right sorted list!


### 📎 **Now use the sort function in the cell below to sort `[(1,5), (7,3), (4,6), (3,2)]` ascending by the first value in the tuple's value in the dictionary `c`.**

In [ ]:
c = {1:10, 3:4, 6:2, 8:9, 2:1, 7:8, 4:10}

# use sort on this list to get the list you came up with above!
sortfn_dict = [(1,5), (7,3), (4,6), (3,2)]
sortfn_dict.sort(reverse=True, key=lambda x:c[x[0]])

In [ ]:
#@title Run this to check your answer!
if sortfn_dict == [(1, 5), (4, 6), (7, 3), (3, 2)]:
  print("That's the right sorted list!")
else:
  print("That's NOT the right sorted list.")

That's the right sorted list!


Now you should have a good idea of how you can use `sort` in your uniform cost search!

<font color="#de3023"><h1><b>STOP #2: Raise your hand to check in with your instructor before moving on.</b></h1></font>


## **Uniform Cost Search**

To get the distance between two nodes, you can use a function that's already been defined for you: `great_circle_distance`. `great_circle_distance` takes in two parameters, both of which are in the form of **(lat, lon)** tuples. The function returns the **distance between those two tuples**, taking into account the curvature of the Earth.

Fill in the function below, using `great_circle_distance` to measure the distance between your nodes. You should be implementing a form of *uniform cost search*!

In [ ]:
def map_shortest_path(nodes, ways, start, goal):
  """
  Return the shortest path between the two locations

  Parameters:
      nodes: the node dictionary
      ways: the ways dictionary
      start: node ID representing the start location
      goal: node ID representing the end location
  Returns:
      a list of node IDs representing the shortest path
      (in terms of distance) from start to goal.
  """
  current = None
  queue = [start]
  came_from = {}
  cost = {start:0}
  visited = set()

  while len(queue)!=0:
    current = queue.pop(0)
    visited.add(current)
    neighbors = ways[current]

    if current == goal:
      path = []
      while current != start:
        path.append(current)
        current = came_from[current]
      path.append(start)
      path.reverse()
      return path

    for n in neighbors:
      if n not in visited: 
        newCost = cost[current] + great_circle_distance(nodes[current],nodes[n])
        if n not in cost or newCost < cost[n]:
          cost[n] = newCost
          came_from[n] = current
          queue.append(n)
    
    queue.sort(reverse=False, key=lambda x:cost[x])

  return None

Once you've coded a solution for your function, make sure you test it out! It should be easiest to test out on the **Stanford** map, which consists of `stanford_nodes` and `stanford_ways` (which you created!). Write **at least two** test cases using the Stanford map below.

In [ ]:
# USE THIS CELL TO WRITE YOUR TEST CASES
print(map_shortest_path(stanford_nodes,stanford_ways,4,9))
print(map_shortest_path(stanford_nodes,stanford_ways,6,9))
print(map_shortest_path(stanford_nodes,stanford_ways,4,7))

[4, 3, 1, 7, 9]
[6, 5, 9]
[4, 3, 1, 7]


<font color="#de3023"><h1><b>STOP #3: Raise your hand to check in with your instructor.</b></h1></font>


# 🏎 **Need for speed!**
If you've ever driven or been in a car, you know that it's not always the physical distance that matters to get you to a location: sometimes we care more about how **long** it will take us to get somewhere, and speed limits on roads exist! 

We're going to change up our earlier code just a bit to incorporate **how much time** (estimated) it will take us to get from node to node as our cost instead of the distance.

## 🔄 **Changes in data**

We're going to add some new information to our data set: speed! For the Bay Area maps, we'll be using `bay_nodes` (same as before) and `bay_speed_ways` (new!). For the Stanford map, we'll be using `stanford_nodes` (same as before) and `stanford_speed_ways` (new!).

In the new ways dictionaries, ways are now represented as node IDs mapped to a list of tuples, where each tuple is (node ID, speed limit) representing a neighboring node and the speed limit on the road between the key and value node.

**Answer the questions below to make sure you understand the new data structure.**

### 🛑 **What is the speed limit on the road between nodes 3 and 10?**

In [ ]:
# use this cell to answer the question!

speed_310 = stanford_speed_ways[3][[i[0] for i in stanford_speed_ways[3]].index(10)][1]

In [ ]:
#@title Run this to check your answer!
if speed_310 == 35:
  print("That's the right speed!")
else:
  print("That's NOT the right speed.")

That's the right speed!


### 🛑 **What is the speed limit on the road between nodes 2 and 8?**

In [ ]:
# use this cell to answer the question!
speed_28 = stanford_speed_ways[2][[i[0] for i in stanford_speed_ways[2]].index(8)][1]

In [ ]:
#@title Run this to check your answer!
if speed_28 == 15:
  print("That's the right speed!")
else:
  print("That's NOT the right speed.")

That's the right speed!


### ⏰ **How much time does it take to travel between nodes 2 and 8?**
**Hint:** remember that you can use `great_circle_distance` to calculate the distance between two nodes. Remember that the given time is in *hours*.

In [ ]:
# use this cell to answer the question!

time_28 = great_circle_distance(stanford_nodes[2],stanford_nodes[8])/stanford_speed_ways[2][[i[0] for i in stanford_speed_ways[2]].index(8)][1]

In [ ]:
#@title Run this to check your answer!
if round(time_28, 4) == 0.0113:
  print("That's the right time!")
else:
  print("That's NOT the right time.")

That's the right time!


## 🏁 **Speedy Paths!**

Now that we know how the data is formatted, let's get to it! We're still going to use `great_circle_distance` to find the distance between nodes, but we'll use the speed limit passed in to make the cost be based on **time**. Remember, you can calculate time by doing: `time = distance / speed`.

You should still be implementing a form of *uniform cost search*!

In [ ]:
def get_time(nodes,speed_ways,node1,node2):
  return great_circle_distance(nodes[node1],nodes[node2])/speed_ways[node1][[i[0] for i in speed_ways[node1]].index(node2)][1]

0.008861891421840127


In [ ]:
def map_speediest_path(nodes, ways, start, goal):
  """
  Return the fastest path between the two locations

  Parameters:
      nodes: the node dictionary
      ways: the ways dictionary
      start: node ID representing the start location
      goal: node ID representing the end location
  Returns:
      a list of node IDs representing the shortest path
      (in terms of time) from start to goal.
  """
  current = None
  queue = [start]
  came_from = {}
  cost = {start:0}
  visited = set()

  while len(queue)!=0:
    current = queue.pop(0)
    visited.add(current)
    neighbors = [i[0] for i in ways[current]]

    if current == goal:
      path = []
      while current != start:
        path.append(current)
        current = came_from[current]
      path.append(start)
      path.reverse()
      return path

    for n in neighbors:
      if n not in visited:
        newCost = cost[current] + get_time(nodes,ways,current,n)
        if n not in cost or newCost < cost[n]:
          cost[n] = newCost
          came_from[n] = current
          queue.append(n)
    
    queue.sort(reverse=False, key=lambda x:cost[x])

  return None

Once you've coded a solution for your function, make sure you test it out! It should be easiest to test out on the **Stanford** map, which consists of `stanford_nodes` and `stanford_speed_ways`. Write **at least two** test cases using the Stanford map below.

In [ ]:
# USE THIS CELL TO WRITE YOUR TEST CASES
print(map_speediest_path(stanford_nodes,stanford_speed_ways,4,9))
print(map_speediest_path(stanford_nodes,stanford_speed_ways,6,9))
print(map_speediest_path(stanford_nodes,stanford_speed_ways,4,7))


[4, 3, 1, 7, 9]
[6, 5, 9]
[4, 3, 1, 7]


<font color="#de3023"><h1><b>STOP #4: Raise your hand to check in with your instructor.</b></h1></font>
